# Caso 3

In [ ]:
from geopy.distance import geodesic
import pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [ ]:
#----- procesamiento de datos/ encontrando las distancias entre todas las locations -----
# https://geopy.readthedocs.io/en/stable/#module-geopy.distance 

clientes = pd.read_csv('Proyecto_C_Caso3/clients.csv')
depositos = pd.read_csv('Proyecto_C_Caso3/depots.csv')
estaciones = pd.read_csv('Proyecto_C_Caso3/stations.csv')
vehiculos = pd.read_csv('Proyecto_C_Caso3/vehicles.csv')

df = pd.read_csv('Proyecto_Caso_Base/locations.csv')
# tomamos el mismo origen y las mismas primeras 15 localidades que en el caso base
df = df[df['LocationID'] < 16]

df.to_csv('Proyecto_C_Caso3/locations.csv', index=False)

for i in range(len(estaciones)):
    data= {'LocationID': estaciones['LocationID'][i],
            'Longitude': estaciones['Longitude'][i],
            'Latitude': estaciones['Latitude'][i],}
    data_f = pd.DataFrame([data])
    # modo a es de append pa ir añadiendo y como ya tiene encabezados header se pone en False
    data_f.to_csv('Proyecto_C_Caso3/locations.csv', mode='a', header=False, index=False)

locations_csv = pd.read_csv('Proyecto_C_Caso3/locations.csv')

locations = []
for i in range(len(locations_csv)):
    latitud = locations_csv['Latitude'][i]
    longitud = locations_csv['Longitude'][i]
    coord = (latitud, longitud)
    locs_i=[]
    j=0
    while j<len(locations_csv):
        latitud2 = locations_csv['Latitude'][j]
        longitud2 = locations_csv['Longitude'][j]
        coord2 = (latitud2, longitud2)

        locs_i.append(geodesic(coord, coord2).meters)
        j+=1
    locations.append(locs_i)

print (locations)


# Formulación matemática corregida del problema

## 1. Formulación del Modelo
Dadas algunas correciones del enunciado y de nuestro modelo original, finalmente se creo este modelo matemático final. Los cambios principales subyacen en el manejo del parámetro de $distancia_{ij}$, y en el uso de las demanadas de cada cliente/municipio. A continuación se explicará cómo fue el modelo que implementamos en este primer caso.


### 1.1. Conjuntos

* $L = \{1, ..., n\}$: Conjunto de localidades (puerto, municipios, estaciones, peajes).
* $D \subseteq L$: Conjunto de destinos/clientes (municipios con demanda).
* $E \subseteq L$: Conjunto de estaciones de recarga.
* $PEA \subseteq L$: Conjunto de localidades con peajes.
* $V = \{1, ..., m\}$: Conjunto de vehículos disponibles.

### 1.2. Índices

* $i, j \in L$: Localidades origen y destino.
* $k \in V$: Vehículo.

### 1.3. Parámetros

* $distancias_{ij}$: Distancia entre localidad $i$ y $j$.
* $D\_demanda\_i$: Demanda del cliente $i \in D$.
* $V\_capacidad\_k$: Capacidad máxima de carga del vehículo $k$.
* $V\_autonomia\_k$: Autonomía máxima del vehículo $k$ (en kilómetros).
* $precio\_combustible\_j$: Precio del galón en estación $j \in E$.
* $costo\_peaje_{ij}$: Costo base del peaje entre $i$ y $j$.
* $peso\_max\_i$: Peso máximo permitido en el municipio $i$.
* $n$: Número total de localidades.

### 1.4. Variables de decisión

* $x_{ijk} \in \{0,1\}$: 1 si el vehículo $k$ viaja de $i$ a $j$, 0 en otro caso.
* $u_{ik} \in \mathbb{Z}$: Orden de visita de $i$ por el vehículo $k$ (usado para eliminar subciclos).
* $r_{jk} \geq 0$: Cantidad de combustible recargado por el vehículo $k$ en la estación $j$.
* $f_{jk} \geq 0$: Nivel de combustible del vehículo $k$ al llegar a $j$.
* $w_{ik} \geq 0$: Peso del vehículo $k$ al ingresar a la localidad $i$.
* $t_{ijk} \in \{0,1\}$: 1 si el vehículo $k$ pasa por un peaje al ir de $i$ a $j$, 0 en otro caso.

---

## 2. Función Objetivo

Minimizar el costo total del recorrido, incluyendo distancia, combustible y peajes:

$$
\min \sum_{k \in V} \left( 
\sum_{\substack{i,j \in L \\ i \ne j}} distancias_{ij} \cdot x_{ijk} 
+ \sum_{j \in E} precio\_combustible_j \cdot r_{jk} 
+ \sum_{\substack{i,j \in L \\ i \ne j}} costo\_peaje_{ij} \cdot w_{ik} \cdot t_{ijk} 
\right)
$$

---

## 3. Restricciones

### (0) Prohibir viajes del puerto a sí mismo:

$$
x_{11k} = 0 \quad \forall k \in V
$$

### (1) Cada cliente debe ser visitado exactamente una vez:

$$
\sum_{k \in V} \sum_{\substack{i \in L \\ i \ne j}} x_{ijk} = 1 \quad \forall j \in D
$$

### (2) Cada vehículo debe salir una sola vez del puerto:

$$
\sum_{j \in L, j \ne 1} x_{1jk} = 1 \quad \forall k \in V
$$

### (3) Cada vehículo debe regresar una vez al puerto:

$$
\sum_{i \in L, i \ne 1} x_{i1k} = 1 \quad \forall k \in V
$$

### (4) Conservación de flujo en los nodos:

$$
\sum_{\substack{i \in L \\ i \ne h}} x_{ihk} = \sum_{\substack{j \in L \\ j \ne h}} x_{hjk} \quad \forall h \in D, \forall k \in V
$$

### (5) Eliminación de subciclos (MTZ):

$$
u_{ik} - u_{jk} + n \cdot x_{ijk} \leq n - 1 \quad \forall i \ne j \in D, \forall k \in V
$$

### (6) Capacidad de cada vehículo:

$$
\sum_{i \in D} D\_demanda\_i \cdot \sum_{\substack{j \in L \\ j \ne i}} x_{ijk} \leq V\_capacidad\_k \quad \forall k \in V
$$

### (7) Autonomía de cada vehículo:

$$
\sum_{\substack{i,j \in L \\ i \ne j}} distancias_{ij} \cdot x_{ijk} \leq V\_autonomia\_k + \sum_{j \in E} r_{jk} \quad \forall k \in V
$$

### (8) Restricción de peso por municipio:

$$
w_{ik} \leq peso\_max\_i \quad \forall i \in D, \forall k \in V
$$

### (9) Actualización del peso transportado:

Si el vehículo $k$ viaja de $i$ a $j$, entonces:

$$
w_{jk} = w_{ik} - D\_demanda\_j \quad \forall i,j \in D, \forall k \in V
$$

### (10) Activación del peaje solo si se transita:

$$
t_{ijk} \leq x_{ijk} \quad \forall i,j \in PEA, \forall k \in V
$$

### (11) Límite superior de variables:

$$
0 \leq r_{jk}, f_{jk}, w_{ik} \leq V\_capacidad\_k \quad \forall i,j \in L, \forall k \in V
$$